In [68]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to C:\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to C:\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [69]:
from nltk.corpus import stopwords 
from nltk.stem.wordnet import WordNetLemmatizer
import string

stopwords = set(stopwords.words('english'))
punctuation = set(string.punctuation) 
lemmatize = WordNetLemmatizer()

def cleaning(article):
    one = " ".join([i for i in article.lower().split() if i not in stopwords])
    two = "".join(i for i in one if i not in punctuation)
    three = " ".join(lemmatize.lemmatize(i) for i in two.split())
    return three

In [70]:
import pandas as pd
df = pd.read_table('march_8.txt', names=['text'])
df.info()
#df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 697 entries, 0 to 696
Data columns (total 1 columns):
text    697 non-null object
dtypes: object(1)
memory usage: 5.5+ KB


In [71]:
df.head(3)

,text
0,bodyText
1,Pemuda Muhammadiyah Minta Pemerintah Jangan Ka...
2,Pemuda Muhammadiyah Minta Pemerintah Jangan Ka...


In [72]:
from nltk.corpus import stopwords 
from nltk.stem.wordnet import WordNetLemmatizer
import string

stopwords = set(stopwords.words('english'))
punctuation = set(string.punctuation) 
lemmatize = WordNetLemmatizer()

def cleaning(article):
    one = " ".join([i for i in article.lower().split() if i not in stopwords])
    two = "".join(i for i in one if i not in punctuation)
    three = " ".join(lemmatize.lemmatize(i) for i in two.split())
    return three

In [73]:
text = df.applymap(cleaning)['text']
type(text)

pandas.core.series.Series

In [74]:
text = df.applymap(cleaning)['text']
text_list = [i.split() for i in text]
len(text_list)

697

In [75]:
type(text)

#text_list[4]

pandas.core.series.Series

In [76]:
from time import time
import logging

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO,
                   filename='running.log',filemode='w')

In [77]:
import gensim
from gensim import corpora

# Creating the term dictionary of our courpus, where every unique term is assigned an index. dictionary = corpora.Dictionary(doc_clean)
dictionary = corpora.Dictionary(text_list)
dictionary.save('dictionary.dict')
print(dictionary)



Dictionary(1670 unique tokens: ['bodytext', 'denganfreeport', 'httpstcos7toq74s1a', 'jangan', 'kalah']...)


In [78]:
doc_term_matrix = [dictionary.doc2bow(doc) for doc in text_list]
corpora.MmCorpus.serialize('corpus.mm', doc_term_matrix)

print(len(doc_term_matrix))
print(doc_term_matrix[100])

697
[(12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1)]


In [79]:
start = time()
# Creating the object for LDA model using gensim library
Lda = gensim.models.ldamodel.LdaModel

# Running and Trainign LDA model on the document term matrix.
ldamodel = Lda(doc_term_matrix, num_topics=5, id2word = dictionary, passes=50)
print('used: {:.2f}s'.format(time()-start))

used: 49.47s


In [80]:
print(ldamodel.print_topics(num_topics=5, num_words=4))

[(0, '0.057*"freeport" + 0.024*"indonesia" + 0.021*"kita" + 0.020*"ahok"'), (1, '0.044*"freeport" + 0.041*"titik" + 0.041*"banjir" + 0.027*"negara"'), (2, '0.054*"freeport" + 0.035*"jakarta" + 0.019*"jokowi" + 0.019*"need"'), (3, '0.056*"freeport" + 0.023*"ahok" + 0.022*"muhammadiyah" + 0.019*"pemerintah"'), (4, '0.060*"freeport" + 0.043*"ahok" + 0.031*"demo" + 0.029*"dan"')]


In [81]:
for i in ldamodel.print_topics(): 
    for j in i: print(j)

0
0.057*"freeport" + 0.024*"indonesia" + 0.021*"kita" + 0.020*"ahok" + 0.016*"jakarta" + 0.015*"jokowi" + 0.014*"sikat" + 0.014*"badan" + 0.013*"pemerintah" + 0.012*"dan"
1
0.044*"freeport" + 0.041*"titik" + 0.041*"banjir" + 0.027*"negara" + 0.024*"indonesia" + 0.024*"dg" + 0.022*"di" + 0.021*"tahun" + 0.021*"dari" + 0.021*"jkt"
2
0.054*"freeport" + 0.035*"jakarta" + 0.019*"jokowi" + 0.019*"need" + 0.017*"ahok" + 0.013*"yang" + 0.012*"harus" + 0.011*"ujung2nya" + 0.010*"turun" + 0.010*"good"
3
0.056*"freeport" + 0.023*"ahok" + 0.022*"muhammadiyah" + 0.019*"pemerintah" + 0.018*"dan" + 0.017*"jangan" + 0.016*"untuk" + 0.015*"pemuda" + 0.015*"soal" + 0.014*"yang"
4
0.060*"freeport" + 0.043*"ahok" + 0.031*"demo" + 0.029*"dan" + 0.019*"lawan" + 0.017*"jokowi" + 0.016*"yg" + 0.016*"tegas" + 0.015*"amerika" + 0.015*"sikap"


In [82]:
ldamodel.save('topic.model')

In [83]:
from gensim.models import LdaModel
loading = LdaModel.load('topic.model')

In [84]:
print(loading.print_topics(num_topics=4, num_words=10))

[(2, '0.054*"freeport" + 0.035*"jakarta" + 0.019*"jokowi" + 0.019*"need" + 0.017*"ahok" + 0.013*"yang" + 0.012*"harus" + 0.011*"ujung2nya" + 0.010*"turun" + 0.010*"good"'), (4, '0.060*"freeport" + 0.043*"ahok" + 0.031*"demo" + 0.029*"dan" + 0.019*"lawan" + 0.017*"jokowi" + 0.016*"yg" + 0.016*"tegas" + 0.015*"amerika" + 0.015*"sikap"'), (1, '0.044*"freeport" + 0.041*"titik" + 0.041*"banjir" + 0.027*"negara" + 0.024*"indonesia" + 0.024*"dg" + 0.022*"di" + 0.021*"tahun" + 0.021*"dari" + 0.021*"jkt"'), (0, '0.057*"freeport" + 0.024*"indonesia" + 0.021*"kita" + 0.020*"ahok" + 0.016*"jakarta" + 0.015*"jokowi" + 0.014*"sikat" + 0.014*"badan" + 0.013*"pemerintah" + 0.012*"dan"')]


In [85]:
def pre_new(doc):
    one = cleaning(doc).split()
    two = dictionary.doc2bow(one)
    return two

In [86]:
pre_new('new article that to be classified by trained model!')

[]

In [87]:
belong = loading[(pre_new('new article that to be classified by trained model!'))]
belong

[(0, 0.2), (1, 0.2), (2, 0.2), (3, 0.2), (4, 0.2)]

In [88]:
new = pd.DataFrame(belong,columns=['id','prob']).sort_values('prob',ascending=False)
new['topic'] = new['id'].apply(loading.print_topic)
new


,id,prob,topic
0,0,0.2,"0.057*""freeport"" + 0.024*""indonesia"" + 0.021*""..."
1,1,0.2,"0.044*""freeport"" + 0.041*""titik"" + 0.041*""banj..."
2,2,0.2,"0.054*""freeport"" + 0.035*""jakarta"" + 0.019*""jo..."
3,3,0.2,"0.056*""freeport"" + 0.023*""ahok"" + 0.022*""muham..."
4,4,0.2,"0.060*""freeport"" + 0.043*""ahok"" + 0.031*""demo""..."


In [89]:
new['topic']

0    0.057*"freeport" + 0.024*"indonesia" + 0.021*"...
1    0.044*"freeport" + 0.041*"titik" + 0.041*"banj...
2    0.054*"freeport" + 0.035*"jakarta" + 0.019*"jo...
3    0.056*"freeport" + 0.023*"ahok" + 0.022*"muham...
4    0.060*"freeport" + 0.043*"ahok" + 0.031*"demo"...
Name: topic, dtype: object

In [90]:
import pyLDAvis.gensim
import gensim
pyLDAvis.enable_notebook()

In [91]:
d = gensim.corpora.Dictionary.load('dictionary.dict')
c = gensim.corpora.MmCorpus('corpus.mm')
lda = gensim.models.LdaModel.load('topic.model')


In [92]:
data = pyLDAvis.gensim.prepare(lda, c, d)
data

PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
1      31.570991        1       1  0.318916  0.006410
0      22.568057        1       2 -0.051018  0.033929
4      22.147099        1       3 -0.087778 -0.222996
2      11.861432        1       4 -0.108809  0.182993
3      11.852422        1       5 -0.071311 -0.000336, topic_info=     Category        Freq                Term       Total  loglift  logprob
term                                                                       
971   Default  150.000000               titik  150.000000  30.0000  30.0000
774   Default  150.000000              banjir  150.000000  29.0000  29.0000
65    Default   78.000000                demo   78.000000  28.0000  28.0000
136   Default   97.000000             jakarta   97.000000  27.0000  27.0000
13    Default  219.000000                ahok  219.000000  26.0000  26.0000
966   Default   86.000000                  dg   86.000000  25.0000  25.0000
822   Default   77.000000               tahun   77.000000  24.0000  24.0000
204   Default   75.000000                 jkt   75.000000  23.0000  23.0000
961   Default   75.000000                2017   75.000000  22.0000  22.0000
963   Default   74.000000                  80   74.000000  21.0000  21.0000
968   Default   74.000000               hujan   74.000000  20.0000  20.0000
960   Default   74.000000                2012   74.000000  19.0000  19.0000
967   Default   74.000000  httpstcoma9nlr3df3   74.000000  18.0000  18.0000
969   Default   74.000000             tersisa   74.000000  17.0000  17.0000
965   Default   74.000000               deras   74.000000  16.0000  16.0000
970   Default   74.000000                 thn   74.000000  15.0000  15.0000
964   Default   74.000000        alhamdulilah   74.000000  14.0000  14.0000
962   Default   74.000000                2200   74.000000  13.0000  13.0000
128   Default  112.000000              negara  112.000000  12.0000  12.0000
54    Default  110.000000              jokowi  110.000000  11.0000  11.0000
16    Default  138.000000                 dan  138.000000  10.0000  10.0000
6     Default   30.000000        muhammadiyah   30.000000   9.0000   9.0000
7     Default   67.000000          pemerintah   67.000000   8.0000   8.0000
17    Default  614.000000            freeport  614.000000   7.0000   7.0000
71    Default   59.000000               lawan   59.000000   6.0000   6.0000
558   Default   27.000000                need   27.000000   5.0000   5.0000
88    Default   39.000000               sikap   39.000000   4.0000   4.0000
83    Default   38.000000               jelas   38.000000   3.0000   3.0000
85    Default   38.000000             polemik   38.000000   2.0000   2.0000
63    Default   38.000000           beraninya   38.000000   1.0000   1.0000
...       ...         ...                 ...         ...      ...      ...
826    Topic5    5.958151             demo2an    6.656183   2.0219  -5.4389
831    Topic5    5.958151             sekelas    6.656183   2.0219  -5.4389
827    Topic5    5.958151                kaum    6.656183   2.0219  -5.4389
828    Topic5    5.958151                mslh    6.656183   2.0219  -5.4389
830    Topic5    5.958151        penghadangan    6.656183   2.0219  -5.4389
829    Topic5    5.958151              peduli    6.656183   2.0219  -5.4389
812    Topic5    5.958134                rais    6.656184   2.0218  -5.4389
808    Topic5    5.958134               amien    6.656184   2.0218  -5.4389
1008   Topic5    5.957536                  hr    6.656224   2.0217  -5.4390
809    Topic5    5.130627  httpstcowc0fcxcefg    5.828662   2.0051  -5.5884
810    Topic5    5.130627                lama    5.828662   2.0051  -5.5884
814    Topic5    5.130627             serukan    5.828662   2.0051  -5.5884
813    Topic5    5.130627               sejak    5.828662   2.0051  -5.5884
18     Topic5    4.303102  httpstco7v6syacnp6    5.001141   1.9823  -5.7643
217    